In [18]:
import os
import sys
import pandas as pd

import rasterio
from rasterio import features
from rasterio.mask import mask
from rasterio.crs import CRS
from shapely.geometry import mapping
from shapely.geometry import shape
import numpy as np
import geopandas as gpd
from scipy import ndimage

from rasterstats import zonal_stats
from rasterio.windows import from_bounds

from shapely.ops import polygonize

tile_of_interest = 'SS79'
wd = 'Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/3_Test_Square'

In [2]:
###############################
# Get extent of tile 
###############################

fp = 'Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/1_Reference_Data/4_Wales_10km_Squares/Wales_10km_Squares.shp'
wales_10km_footprint = gpd.read_file(fp)

tile_footprint = wales_10km_footprint[wales_10km_footprint['tileref'] == tile_of_interest]
tile_footprint = tile_footprint.set_crs('EPSG:27700') 

In [ ]:
#####################################
# Remove pixels below 2.5m from CHM
#####################################

print('Removing pixels below 2.5m from CHM')

chm_fp = 'Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/1_Reference_Data/1_Wales_LiDAR/Wales_CHM_2020_22_32bit.tif'

with rasterio.open(chm_fp) as src:

    window = from_bounds(*tile_footprint.total_bounds, transform=src.transform)
    chm_data = src.read(1, window=window).astype('float32')

    out_transform = src.window_transform(window)
    out_meta = src.meta.copy()
    chm_crs = src.crs
    
    chm_data = np.where(chm_data < 1.3, np.nan, chm_data)
    
out_shape = (int(window.height), int(window.width))

Removing pixels below 2.5m from CHM


In [1]:
#################################
# Create dictionary OSMM terms
#################################

osm_desc_terms = pd.read_csv('Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/1_Reference_Data/5_Wales_OSMM_2023/descterm_categories.csv')
osm_desc_terms.columns = osm_desc_terms.columns.str.strip()
osm_desc_terms = osm_desc_terms.map(lambda x: x.strip() if isinstance(x, str) else x)

osm_terms = {
    "tree_terms": osm_desc_terms.loc[osm_desc_terms["Term (ED)"] == "tree_term", "Descterm"].tolist(),
    "mask_terms": osm_desc_terms.loc[osm_desc_terms["Term (ED)"] == "mask_term", "Descterm"].tolist(),
    "structure_terms": osm_desc_terms.loc[osm_desc_terms["Term (ED)"] == "structure_term", "Descterm"].tolist(),
    "water_terms": osm_desc_terms.loc[osm_desc_terms["Term (ED)"] == "water_body", "Descterm"].tolist()
}

for k, v in osm_terms.items():
    print(f"{k}: {v}")

NameError: name 'pd' is not defined

In [ ]:
##############################
# Categorise OSMM
##############################

print('Categorising OSMM features')

OSMM_path = f'{wd}/OSMM_Square_SS79/OSMM_Square_SS79.shp'
OSMM = gpd.read_file(OSMM_path)

if OSMM.crs != chm_crs:
    OSMM = OSMM.to_crs(chm_crs)

# Update features with blank 'descterm' field
OSMM["descterm"].replace('', np.nan, inplace=True)

for value in OSMM["descgroup"].unique():
    OSMM.loc[
        (OSMM["descterm"].isna()) & (OSMM["descgroup"] == value),
        "descterm"
    ] = value

# Create Surf_Obj field
OSMM["Surf_Obj"] = None

#tree_terms = ["Agricultural Land", "Aqueduct,Scrub", "Boulders (Scattered),Coniferous Trees,Nonconiferous Trees,Scrub", "Boulders (Scattered),Nonconiferous Trees", "Boulders (Scattered),Rough Grassland,Scrub", "General Surface", "Bridge,Nonconiferous Trees", "Bridge,Nonconiferous Trees,Scrub", "Bridge,Rough Grassland", "Canal", "Cliff", "Collects", "Coniferous Trees", "Coniferous Trees (Scattered)", "Coniferous Trees (Scattered),Nonconiferous Trees (Scattered)", "Coniferous Trees (Scattered),Nonconiferous Trees (Scattered),Scrub", "Coniferous Trees (Scattered),Rough Grassland", "Coniferous Trees (Scattered),Rough Grassland,Scrub", "Coniferous Trees (Scattered),Scrub", "Coniferous Trees,Mineral Workings (Inactive)", "Coniferous Trees,Nonconiferous Trees", "Coniferous Trees,Nonconiferous Trees,Scrub", "Coniferous Trees,Scrub", "Coniferous Trees,Scrub,Nonconiferous Trees", "Coniferous Trees,Static Water", "Coppice Or Osiers,Nonconiferous Trees", "Drain", "Ford", "Heath", "Heath,Nonconiferous Trees (Scattered)", "Heath,Nonconiferous Trees (Scattered),Rough Grassland", "Heath,Rough Grassland", "Heath,Rough Grassland,Scrub", "Inland Water", "Landform", "Marsh", "Marsh,Nonconiferous Trees", "Marsh,Nonconiferous Trees,Scrub", "Marsh,Rough Grassland", "Marsh,Rough Grassland,Scrub", "Marsh,Scrub", "Mineral Workings", "Mineral Workings (Inactive)", "Mineral Workings (Inactive),Nonconiferous Trees", "Mineral Workings (Inactive),Nonconiferous Trees,Scrub", "Mineral Workings (Inactive),Slope", "Mud", "Multi Surface", "Nonconiferous Trees", "Nonconiferous Trees (Scattered)", "Nonconiferous Trees (Scattered),Heath", "Nonconiferous Trees (Scattered),Rough Grassland", "Nonconiferous Trees (Scattered),Rough Grassland,Scrub", "Nonconiferous Trees (Scattered),Scrub", "Nonconiferous Trees (Scattered),Scrub,Rough Grassland", "Nonconiferous Trees,Boulders", "Nonconiferous Trees,Coniferous Trees", "Nonconiferous Trees,Coniferous Trees,Scrub", "Nonconiferous Trees,Coppice Or Osiers", "Nonconiferous Trees,Scrub", "Nonconiferous Trees,Scrub,Coniferous Trees", "Orchard", "Path", "Path,Structure", "Rail", "Reeds,Static Water", "Reeds,Watercourse", "Road Or Track", "Roadside", "Rock", "Rough Grassland", "Rough Grassland,Boulders", "Rough Grassland,Boulders,Heath", "Rough Grassland,Coniferous Trees (Scattered)", "Rough Grassland,Heath", "Rough Grassland,Nonconiferous Trees (Scattered)", "Rough Grassland,Sand", "Rough Grassland,Scrub", "Rough Grassland,Scrub,Boulders (Scattered)", "Rough Grassland,Scrub,Nonconiferous Trees (Scattered)", "Rough Grassland,Scrub,Spoil Heap (Inactive)", "Scrub", "Scrub,Coniferous Trees", "Scrub,Coniferous Trees,Nonconiferous Trees", "Scrub,Nonconiferous Trees", "Scrub,Nonconiferous Trees (Scattered)", "Scrub,Nonconiferous Trees,Coniferous Trees", "Scrub,Rough Grassland", "Scrub,Rough Grassland,Nonconiferous Trees (Scattered)", "Scrub,Spoil Heap (Inactive)", "Shingle", "Sinks", "Slipway", "Slope", "Sloping Masonry", "Spring", "Step", "Track", "Traffic Calming", "Unclassified", "Waterfall", "Weir"]
OSMM.loc[OSMM["descterm"].isin(osm_terms['tree_terms']), "Surf_Obj"] = "No"

#surface_terms = ["Archway", "Aqueduct,Canal", "Bridge", "Bridge,Step", "Building", "Building,Structure", "Chimney", "Conveyor", "Conveyor,Overhead Construction", "Gas Governor", "Electricity Sub Station", "Footbridge", "Footbridge,Step", "Fountain", "Gantry", "General Surface,Structure", "General Surface,Rail", "General Surface,Roadside,Structure", "Glasshouse", "Level Crossing", "Public Convenience", "Pylon", "Rail Signal Gantry", "Rail,Structure", "Roadside,Structure", "Structure", "Structure,Inland Water", "Tank"]
OSMM.loc[OSMM["descterm"].isin(osm_terms['structure_terms']), "Surf_Obj"] = "Yes"

#mask_terms = ["Boulders", "Boulders,Foreshore", "Foreshore", "Foreshore,Mud", "Foreshore,Mud,Sand", "Foreshore,Mud,Shingle", "Foreshore,Saltmarsh", "Foreshore,Sand", "Foreshore,Shingle", "Foreshore,Slipway", "Foreshore,Sloping Masonry", "Mud,Sand", "Sand", "Swimming Pool", "Tidal Water"]
OSMM.loc[OSMM["descterm"].isin(osm_terms['mask_terms']), "Surf_Obj"] = "Mask"

#water_terms = ["Reservoir", "Static Water", "Watercourse"]
OSMM.loc[OSMM["descterm"].isin(osm_terms['water_terms']), "Surf_Obj"] = "Water"

Categorising OSMM features


C:\Users\eleanor.downer\AppData\Local\Temp\ipykernel_12136\1387477486.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  OSMM["descterm"].replace('', np.nan, inplace=True)


In [12]:
######################################
# Mask water bodies over 500m2
######################################

print('Masking water bodies over 500m2')

water_gdf = OSMM[OSMM["Surf_Obj"] == "Water"].copy()
large_water_gdf = water_gdf[water_gdf['calcarea'] > 500]
water_gdf = water_gdf.to_crs(chm_crs)

# Rasterise mask_terms
water_mask = features.rasterize(
    [(geom, 1) for geom in large_water_gdf.geometry],
    out_shape=out_shape,
    transform=out_transform,
    fill=0,
    dtype="uint8"
)

# Apply water mask to CHM
chm_data = np.where(water_mask == 1, np.nan, chm_data)

Masking water bodies over 500m2


In [13]:
###########################################
# Mask out non-tree OSMM features from CHM
###########################################

print('Masking out non-tree OSMM features from CHM')

# Buffer and reproject surface_terms
surface_gdf = OSMM[OSMM["Surf_Obj"] == "Yes"].copy()
surface_gdf = surface_gdf.to_crs(chm_crs)
surface_gdf["geometry"] = surface_gdf.buffer(2)

# Get mask_terms and reproject
mask_gdf = OSMM[OSMM["Surf_Obj"] == "Mask"].copy()
mask_gdf = mask_gdf.to_crs(chm_crs)

# Rasterise buffered surface_terms
surface_mask = features.rasterize(
        [(geom, 1) for geom in surface_gdf.geometry],
        out_shape=out_shape,
        transform=out_transform,
        fill=0,
        dtype="uint8"
)

# Rasterise mask_terms
mask_mask = features.rasterize(
        [(geom, 1) for geom in mask_gdf.geometry],
        out_shape=out_shape,
        transform=out_transform,
        fill=0,
        dtype="uint8"
)

# Apply both masks
combined_mask = (surface_mask == 1) | (mask_mask == 1)
chm_data[combined_mask] = np.nan

Masking out non-tree OSMM features from CHM


In [14]:
###########################################
# Mask out buildings from OpenStreetMap
###########################################

print('Masking out buildings from OpenStreetMap')

osm_fp = f'C:/Users/eleanor.downer/OneDrive - Forest Research/Documents/TOW_Wales/wales_osm_clean.gpkg'
osm_polygons = gpd.read_file(osm_fp, layer='multipolygons', bbox=tile_footprint)
osm_buildings = osm_polygons[osm_polygons['building'].notna()].copy()
osm_buildings = osm_buildings.to_crs(chm_crs)

# Rasterise buildings
osm_buildings_mask = features.rasterize(
    [(geom, 1) for geom in osm_buildings.geometry],
    out_shape=out_shape,
    transform=out_transform,
    fill=0,
    dtype="uint8"
)

osm_buildings_mask = osm_buildings_mask.astype(bool)
chm_data[osm_buildings_mask] = np.nan

Masking out buildings from OpenStreetMap


c:\PythonWorkspaces\VirtualEnvironment\Lib\site-packages\geopandas\io\file.py:484: UserWarning: More than one layer found in 'wales_osm_clean.gpkg': 'points' (default), 'lines', 'multilinestrings', 'other_relations', 'multipolygons'. Specify layer parameter to avoid this warning.
  crs = pyogrio.read_info(path_or_bytes).get("crs")


In [15]:
###########################################
# Mask power cables
###########################################

print('Masking out power cables from CHM')

cables_fp = 'Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/1_Reference_Data/10_Power_Lines/OHL_buffered.gpkg'
cables = gpd.read_file(cables_fp)
cables = cables.set_crs(chm_crs) 

# Rasterise cables
cables_mask = features.rasterize(
    [(geom, 1) for geom in cables.geometry],
    out_shape=out_shape,
    transform=out_transform,
    fill=0,
    dtype="uint8"
)

cables_mask = cables_mask.astype(bool)

over_20m_mask = chm_data > 20
cables_mask = cables_mask & over_20m_mask

chm_data[cables_mask] = np.nan

Masking out power cables from CHM


In [ ]:
###########################################
# Mask power structures from OpenStreetMap
###########################################

print('Masking out power structures from CHM')

power_structures_fp = 'Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/1_Reference_Data/11_OpenStreetMap/osm_power_structures_v2.gpkg'
power_structures = gpd.read_file(power_structures_fp, bbox=tile_footprint)

power_structures = power_structures[power_structures['power_source'] != 'solar'].copy()

if power_structures.crs != chm_crs:
    power_structures = power_structures.to_crs(chm_crs)

power_structures_mask = features.rasterize(
    [(geom, 1) for geom in power_structures.geometry],
    out_shape=out_shape,
    transform=out_transform,
    fill=0,
    dtype="uint8"
)

power_structures_mask = power_structures_mask.astype(bool)
chm_data[power_structures_mask] = np.nan

Masking out power structures from CHM


In [17]:
####################################
# Remove clusters smaller than 5m2
####################################

# Create binary mask of valid pixels
mask = ~np.isnan(chm_data)

structure = ndimage.generate_binary_structure(2, 1) # Label connected components
labeled, num = ndimage.label(mask, structure=structure)
sizes = np.bincount(labeled.ravel()) # Count component sizes

# Build keep mask
keep_mask = np.zeros_like(sizes, dtype=bool)
keep_mask[sizes >= 5] = True
keep_mask[0] = False  # background always False

chm_data = np.where(keep_mask[labeled], chm_data, np.nan)

In [18]:
####################################
# Save to file - with NFI still
####################################

out_meta.update({
    'width' : out_shape[1],
    'height' : out_shape[0],
    'crs' : chm_crs,
    'transform' : out_transform,
})

chm_data = np.where(np.isnan(chm_data), -9999.0, chm_data)

output_path = f'{wd}/VOM/{tile_of_interest}_VOM_with_NFI.tif'
with rasterio.open(output_path, "w", **out_meta) as dst:
    dst.write(chm_data, 1)

Hedge extraction here.

Raster of CHM with hedges removed.
tif saved here : //forestresearch.gov.uk/shares/IFOS/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/3_Test_Square/Tests/SS79_VOM_hedges_filtered.tif

Height variation mask also created in this step.

In [44]:
#############################################
# Load in CHM after hedge extraction process
#############################################

chm_path = f'{wd}/VOM/{tile_of_interest}_VOM_NFI_hedges_extracted.tif'

with rasterio.open(chm_path) as src:
    chm_data = src.read(1).astype('float32')
    chm_transform = src.transform
    chm_meta = src.meta
    chm_crs = src.crs

In [ ]:
print(chm_data.shape)

In [ ]:
#############################################
# Load in height variation raster
#############################################

################################################
# Mask out large, flat objects
# (flat buildings/structures and gorse/shrub)
################################################

chm_var_path = f"{wd}/VOM/height_var/{tile_of_interest}_VOM_NFI_hedges_extracted_hv3x3.tif"

with rasterio.open(chm_var_path) as var_src:
    chm_var_data = var_src.read(1).astype(np.float16)
    chm_var_data = np.where(chm_var_data > 0.3, np.nan, chm_var_data)
    var_meta = var_src.meta.copy()

    chm_crs = var_src.crs

mask = ~np.isnan(chm_var_data) # Create binary mask of valid pixels

structure = ndimage.generate_binary_structure(2, 1) # Label connected components
labeled, num = ndimage.label(mask, structure=structure)
sizes = np.bincount(labeled.ravel()) # Count component sizes

# Build keep mask
keep_mask = np.zeros_like(sizes, dtype=bool)
keep_mask[sizes >= 100] = False
keep_mask[0] = True

chm_var_data = np.where(keep_mask[labeled], chm_var_data, np.nan)

mask = (~np.isnan(chm_var_data))
mask = mask.astype('int16')



# Mask out low variation areas from VOM
chm_data[mask] = np.nan

(9999, 9999)


In [33]:
####################################
# Remove pixels lower than 2.5m
####################################

chm_data[chm_data < 2.5] = np.nan

In [34]:
####################################
# Mask out NFI Woodland
####################################

print('Masking out NFI Woodland from CHM')

NFI_woodland_fp = 'Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/1_Reference_Data/6_Wales_NFI_2023/NFI_Wales_2023_WoodlandOnly.gpkg'
nfi_gdf = gpd.read_file(NFI_woodland_fp, bbox=tile_footprint)

# Reproject NFI to match CHM 
if nfi_gdf.crs != chm_crs:
    nfi_gdf = nfi_gdf.to_crs(chm_crs)

# Rasterise NFI
nfi_mask = features.rasterize(
    [(geom, 1) for geom in nfi_gdf.geometry],
    out_shape=chm_data.shape, #out_shape=out_shape,
    transform=chm_transform, #transform=out_transform,
    fill=0,
    dtype="uint8"
)

# Mask NFI out of VOM
nfi_mask = nfi_mask.astype(bool)
chm_data[nfi_mask] = np.nan

Masking out NFI Woodland from CHM


In [ ]:
####################################
# NDVI Thresholding (0.2)
####################################

ndvi_fp = 'Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/3_Test_Square/NDVI/SS79_ndvi_composite_resampled_v3.tif'
with rasterio.open(ndvi_fp) as src:
    ndvi_array = src.read(1).astype('float32')

ndvi_array[ndvi_array == -9999] = np.nan
ndvi_mask = ndvi_array < 0.2

#chm_data[ndvi_mask] = np.nan

print(ndvi_mask.shape)
print(chm_data.shape)

(10000, 10000)
(9999, 9999)


In [38]:
####################################
# NDVI Thresholding (2023 Update)
####################################
from skimage import morphology
from rasterio.features import shapes

thresh = 0.5
mask_data = np.zeros(ndvi_array.shape)
mask_data[ndvi_array <= thresh] = 1 # Set pixels with mean yearly NDVI greater than threshold to 1
mask_data = mask_data.astype(np.int16)  # Should print [0, 1]

shrink_size = 3  # Number of pixels to shrink by
selem = morphology.disk(shrink_size)  # Create a disk-shaped element for erosion
shrunken_mask = morphology.erosion(mask_data, selem)

print(shrunken_mask.shape)

NameError: name 'ndvi_array' is not defined

In [35]:
####################################
# Remove clusters smaller than 5m2
####################################

# Create binary mask of valid pixels
mask = ~np.isnan(chm_data)

structure = ndimage.generate_binary_structure(2, 1) # Label connected components
labeled, num = ndimage.label(mask, structure=structure)
sizes = np.bincount(labeled.ravel()) # Count component sizes

# Build keep mask
keep_mask = np.zeros_like(sizes, dtype=bool)
keep_mask[sizes >= 5] = True
keep_mask[0] = False  # background always False

chm_data = np.where(keep_mask[labeled], chm_data, np.nan)

In [36]:
####################################
# Save to file
####################################

chm_meta.update({
    'width' : (chm_data.shape)[1], # out_shape[1],
    'height' : (chm_data.shape)[0], # out_shape[0],
    'crs' : chm_crs,
    'transform' : chm_transform # out_transform,
})

chm_data = np.where(np.isnan(chm_data), -9999.0, chm_data)

output_path = f'{wd}/VOM/{tile_of_interest}_VOM_final.tif'
with rasterio.open(output_path, "w", **chm_meta) as dst:
    dst.write(chm_data, 1)